In [96]:
# Cryptanalysis - Task 3 - Truncated attack on CipherFour - v1

# TO-DO:

# Remarks:


import numpy as np

def sbox(input):
    list_out = [6, 4, 12, 5, 0, 7, 2, 14, 1, 15, 3, 13, 8, 10, 9, 11]
    return list_out[input]

def inverse_sbox(input):
    list_out = [6, 4, 12, 5, 0, 7, 2, 14, 1, 15, 3, 13, 8, 10, 9, 11]
    return list_out.index(input)

def sbox_layer(input):
    out = []
    for i in range(4):
        #print((input >> (i * 4) % 16))
        out.append(sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def sbox_layer_inverse(input):
    out = []
    for i in range(4):
        #print(((input >> (i * 4)) % 16))
        out.append(inverse_sbox(((input >> (i * 4)) % 16)))
    res = 0
    for i in range(4):
        res += out[i] * (16 ** i)
    return res

def permutation(inputs):
    inp = inputs
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm[15 - i]] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)

def inverse_permutation(input):
    inp = input
    perm = [0, 4, 8, 12, 1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15]
    arr = [0] * 16
    for i in range(16):
        if inp % 2 == 1:
            arr[perm.index(15 - i)] = 1
        inp = inp >> 1
    return int("".join(str(x) for x in arr), 2)


def encrypt(keys, message):
    ct = message
    for i in range(len(keys) - 2):
        ct ^^= keys[i]
        ct = sbox_layer(ct)
        ct = permutation(ct)
    ct ^^= keys[-2]
    ct = sbox_layer(ct)
    ct ^^= keys[-1]
    return ct

def decrypt(keys, ct):
    pt = ct
    pt ^^= keys[-1]
    pt = sbox_layer_inverse(pt)
    pt ^^= keys[-2]
    for i in range(len(keys) - 3, -1, -1):
        pt = inverse_permutation(pt)
        pt = sbox_layer_inverse(pt)
        pt ^^= keys[i]
    return pt

def attack(keys):
    # first round trick:  
    # diff = S_box(i ^^ key_guess) ^^ S_box(j ^^ key_guess)
    # => j = key_guess ^^ S_box_inverse(diff ^^ S_box(i ^^ key_guess))
    t_0, t_1, t_2 = randint(0, 15), randint(0, 15), randint(0, 15)
    u_0, u_1, u_2 = randint(0, 15), randint(0, 15), randint(0, 15)
    v_0, v_1, v_2 = randint(0, 15), randint(0, 15), randint(0, 15)
    #print("t:", t_0, t_1, t_2)
    i_s = range(16)
    
    diff_part = 2
    #diff_part = 8
    
    guesses_k0_part = range(16)
    guesses_k4_part = range(16)
    
    final_guesses = []
    for k0_part in guesses_k0_part:
        #print("guessing 4 bits of k0:", k0_part)
        pairs = []
        for i in i_s:
            mes_0 = t_0 * 16**3 + t_1 * 16**2 + i * 16 + t_2
            mes_2 = u_0 * 16**3 + u_1 * 16**2 + i * 16 + u_2
            mes_4 = v_0 * 16**3 + v_1 * 16**2 + i * 16 + v_2
            #mes_prime = t_0 * 16**3 + t_1 * 16**2 + (m ^^ 2) * 16 + t_2
            #mes_prime = t_0 * 16**3 + t_1 * 16**2 + (m ^^ 15) * 16 + t_2

            j = k0_part ^^ inverse_sbox(diff_part ^^ sbox(i ^^ k0_part))
            
            #check_diff = sbox(i ^^ k0_part) ^^ sbox(j ^^ k0_part)
            
            #check_diff_corr = sbox(i ^^ 4) ^^ sbox(j ^^ 4)
            #print("correct:", diff_part, "S(i ^^ guess) ^^ S(j ^^ guess):", check_diff)
            #print("i,j:", i, j, "with k0 guess:", check_diff, "with k0 correct:", check_diff_corr)
            mes_1 = t_0 * 16**3 + t_1 * 16**2 + j * 16 + t_2
            mes_3 = u_0 * 16**3 + u_1 * 16**2 + j * 16 + u_2
            mes_5 = v_0 * 16**3 + v_1 * 16**2 + j * 16 + v_2
            
            
            enc_0 = encrypt(keys, mes_0)
            enc_1 = encrypt(keys, mes_1)
            
            enc_2 = encrypt(keys, mes_2)
            enc_3 = encrypt(keys, mes_3)
            
            enc_4 = encrypt(keys, mes_4)
            enc_5 = encrypt(keys, mes_5)
            
            #k4_real = 5 + 6*16 + 7*16**2 + 8*16**3
            #check_diff = sbox_layer_inverse(enc_0 ^^ k4_real) ^^ sbox_layer_inverse(enc_1 ^^ k4_real)
            #print("should be *0** *0** *0** *0**:", bin(check_diff))
            
            #check_diff = permutation(sbox_layer(mes_0 ^^ keys[0])) ^^ permutation(sbox_layer(mes_1 ^^ keys[0]))
            #check_diff = sbox_layer(mes_0 ^^ keys[0]) ^^ sbox_layer(mes_1 ^^ keys[0])
            #k0_real_part = 4
            #check_diff = sbox(i ^^ k0_real_part) ^^ sbox(j ^^ k0_real_part)
            #print(bin(check_diff))
            
            #pairs.append(((mes_0, enc_0), (mes_1, enc_1)))
            pairs.append((enc_0, enc_1))
            pairs.append((enc_2, enc_3))
            pairs.append((enc_4, enc_5))
            
            #enc_diff = enc_0 ^^ enc_1
            #print(enc_diff)
        
        possible_nibbles = [[], [], [], []]
        
        k0_wrong = False
        for nibble in range(4):
            #print("guessing nibble:", nibble)
            counts = [0]*len(guesses_k4_part)
            
            for pair in pairs:
                x_0, x_1 = pair
                #print(bin(x_0), bin(x_1))
                x_0_part = (x_0 >> ((3 - nibble) * 4)) % 16
                x_1_part = (x_1 >> ((3 - nibble) * 4)) % 16
                #print(bin(x_0_part), bin(x_1_part))
                
                #k4_real = 5 + 6*16 + 7*16**2 + 8*16**3
                #check_diff = sbox_layer_inverse(x_0 ^^ k4_real) ^^ sbox_layer_inverse(x_1 ^^ k4_real)
                #print("should be *0** *0** *0** *0**:", bin(check_diff))
                
                #check_corr = inverse_sbox(x_0_part ^^ 8) ^^ inverse_sbox(x_1_part ^^ 8)
                #check_corr = inverse_sbox(x_0_part ^^ 5) ^^ inverse_sbox(x_1_part ^^ 5)
                #print("corr:", bin(check_corr))
                for k4_part in guesses_k4_part:
                    #print("guess of k4 part:", k4_part)
                    #y_0, y_1 = sbox_layer_inverse(x_0 ^^ k4_part), sbox_layer_inverse(x_1 ^^ k4_part)
                    y_0_part, y_1_part = inverse_sbox(x_0_part ^^ k4_part), inverse_sbox(x_1_part ^^ k4_part)
                    difference = y_0_part ^^ y_1_part
                    #print(bin(diff_part))
                    #print(difference)
                    #bit = ((diff >> 2) >> (3 - nibble)) % 2
                    bit = (difference >> 2) % 2
                    #print(bit)
                    
                    #if nibble == 0 and k4_part == 8:
                    #    placeholder = 0
                    #    print(bit)
                    
                    if bit == 0:
                        counts[k4_part] += 1
                #print("sub-counts:", counts)
            #print("counts:", counts)
            if max(counts) != len(pairs):
                k0_wrong = True
                print("eliminated")
                break
            else:
                for k4_part in guesses_k4_part:
                    if counts[k4_part] == len(pairs):
                        possible_nibbles[nibble].append(k4_part)
                #print("current nibble guesses:", possible_nibbles[nibble])
        
        if not k0_wrong:
            final_guesses.append((k0_part, possible_nibbles))
    
    return final_guesses
                    
            
    #print(encs[0] ^^ encs[1], encs[1] ^^ encs[2])
        


mes_1 = 67

key_0_part = 2
key_0 = 1 + key_0_part*16 + 3*16**2 + 4*16**3
#key_0 = key_0_part*16
key_4_nibbles = [8, 7, 6, 5]
key_4 = key_4_nibbles[3] + key_4_nibbles[2]*16 + key_4_nibbles[1]*16**2 + key_4_nibbles[0]*16**3
#key_4 = [0, 0, 0, 7]
k_1, k_2, k_3 = randint(0, 16**4 - 1), randint(0, 16**4 - 1), randint(0, 16**4 - 1)
#keys_1 = [1 + key_0_part*16 + 3*16**2 + 4*16**3, 2, 3, 4, key_4[3] + key_4[2]*16 + key_4[1]*16**2 + key_4[0]*16**3]
#keys_1 = [key_0_part*16, 2, 3, 4, key_4[3] + key_4[2]*16 + key_4[1]*16**2 + key_4[0]*16**3]

keys_1 = [key_0, k_1, k_2, k_3, key_4]
#keys_1 = [1 + 2*16 + 3*16**2 + 4*16**3, 2, 3, 4, 8*16**3]
#keys_1 = [1 + 2*16 + 3*16**2 + 4*16**3, 2, 3, 4, 5]
enc_1 = encrypt(keys_1, mes_1)
dec_1 = decrypt(keys_1, enc_1)

print("Enc and Dec:", mes_1, "->", enc_1, "->", dec_1)

print("keys:", key_0, k_1, k_2, k_3, key_4)
result = attack(keys_1)
print("correct: k0 part -", key_0_part, "k4 nibbles -", key_4_nibbles)
for i in range(len(result)):
    print("k0 part:", result[i][0], "k4 nibbles:", result[i][1])
#print(attack(keys_1))

Enc and Dec: 67 -> 28829 -> 67
keys: 17185 28847 16142 11465 34661
correct: k0 part - 2 k4 nibbles - [8, 7, 6, 5]
k0 part: 0 k4 nibbles: [[8, 14], [7, 9, 11], [6], [5]]
k0 part: 1 k4 nibbles: [[8, 14], [7, 9, 11], [6], [5]]
k0 part: 2 k4 nibbles: [[0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 14], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [6], [5]]
k0 part: 3 k4 nibbles: [[0, 8, 9, 14], [7, 9, 11], [6], [5]]
k0 part: 4 k4 nibbles: [[8], [7], [5, 6], [5]]
k0 part: 5 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 6 k4 nibbles: [[7, 8], [7], [6], [5]]
k0 part: 7 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 8 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 9 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 10 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 11 k4 nibbles: [[8], [7], [6], [5]]
k0 part: 12 k4 nibbles: [[0, 8, 9, 14], [7], [6], [5]]
k0 part: 13 k4 nibbles: [[8], [7, 9, 11], [6], [5]]
k0 part: 14 k4 nibbles: [[0, 8, 9], [7, 9], [6], [5]]
k0 part: 15 k4 nibbles: [[8, 14], [7, 11], [6], [5]]
